In [ ]:
!pip install vit-keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, Input
from tensorflow.keras.layers import Activation, Concatenate, Conv2D, Multiply
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from vit_keras import vit

warnings.filterwarnings('ignore')
print('TensorFlow Version ' + tf.__version__)

train=r'/content/drive/MyDrive/Dataset with Code/Train'
test=r'/content/drive/MyDrive/Dataset with Code/Test'
val=r'/content/drive/MyDrive/Dataset with Code/Val'

train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                              samplewise_center = True,
                                                              samplewise_std_normalization = True)
val_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                            samplewise_center = True,
                                                            samplewise_std_normalization = True)
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                             samplewise_center = True,
                                                             samplewise_std_normalization = True)

train_ds = train_datagen.flow_from_directory(train,
                                          target_size = (224,224),batch_size = 8,
                                         shuffle = True,seed = 42)
test_ds = test_datagen.flow_from_directory(test,
                                          target_size = (224,224),batch_size = 8,
                                         shuffle = False,seed = 42)
val_ds = val_datagen.flow_from_directory(val,
                                          target_size = (224,224),batch_size = 8,
                                         shuffle = False,seed = 42)

from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.models import Model

vit_model = vit.vit_b16(
        image_size = 224,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 39)

IMAGE_SIZE = 224
initializer = tf.keras.initializers.GlorotNormal()

inputs = Input(shape=(224, 224, 3))
x = vit_model.output
x = Flatten()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(128, activation='relu', kernel_initializer=initializer)(x)
outputs = Dense(39, activation='softmax')(x)
model = Model(inputs = vit_model.input, outputs = outputs, name='vision_transformer_cbam')
#model.summary()

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True)

# Define Reduce Learning Rate Callback
reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    patience=2,
    factor=0.1,
    verbose=1)

# Define Callbacks and Metrics lists
CALLBACKS = [early_stopping_callback, reduce_lr_callback]
METRICS = ['accuracy']

learning_rate = 1e-4
#Initially set with 0.01
# After patience = 5 of val_loss, factor = 0.1

optimizer = tfa.optimizers.RectifiedAdam(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.CategoricalCrossentropy(), # Remove label_smoothing
              metrics=['accuracy'])

history = model.fit(
    train_ds,
    batch_size=8,
    epochs=10,
    validation_data=val_ds,
    callbacks=[early_stopping_callback, reduce_lr_callback]
)

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
import numpy as np

class_labels = list(test_ds.class_indices.keys())
y_pred1 = model.predict(test_ds)
y_pred1 = np.argmax(y_pred1, axis=1)

from sklearn.metrics import classification_report
print(classification_report(test_ds.classes, y_pred1, target_names=class_labels))

In [ ]:
from sklearn.metrics import accuracy_score

# Make predictions on the test data
predictions = model.predict(test_ds)

# Convert predictions and true labels into class indices
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_ds.classes

# Get class labels from the generator
class_labels = list(test_ds.class_indices.keys())

# Calculate accuracy for each class
class_accuracies = {}
for i, label in enumerate(class_labels):
    class_indices = np.where(true_classes == i)[0]
    class_predictions = predicted_classes[class_indices]
    class_true_labels = true_classes[class_indices]
    class_accuracy = accuracy_score(class_true_labels, class_predictions)
    class_accuracies[label] = class_accuracy

# Print accuracy for each class
for label, accuracy in class_accuracies.items():
    print(f"Accuracy for class {label}: {accuracy}")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_true = test_ds.classes

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Assuming labels contain the class names
labels = test_ds.class_indices.keys()

plt.figure(figsize=(18, 18))
hm = sns.heatmap(confusion_matrix(y_true, y_pred1), annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False,
            xticklabels=labels, yticklabels=labels, color='green')
hm.set(xlabel='Predicted_labels')
hm.set(ylabel='True_labels')
plt.show()

In [ ]:
y_true = test_ds.classes

from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred1)

num_classes = cm.shape[0]

specificities = []
for i in range(num_classes):
    tn = cm[i, i]
    fp = cm.sum(axis=0)[i] - tn
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificities.append(specificity)

class_names = test_ds.class_indices.keys()  # Assuming test_data is your test dataset

print("Specificity per class:")
for class_name, class_specificity in zip(class_names, specificities):
    print(f"{class_name}: {class_specificity:.2f}")

class_weights = np.bincount(y_true) / len(y_true)

weighted_specificity = np.average(specificities, weights=class_weights)
print("Weighted Averaged Specificity:", weighted_specificity)

macro_specificity = sum(specificities) / len(specificities)
print("Macro-Averaged Specificity:", macro_specificity)